In [1]:
# Set up key libraries / tokens

In [2]:
import requests 
from requests_oauthlib import OAuth1
import os
import pandas as pd

In [3]:
API_PUBLIC = os.environ.get("TW_API_KEY")
API_SECRET = os.environ.get("TW_API_SECRET_KEY")
APP_PUBLIC = os.environ.get("TW_PRS_GA_HW_ACCESS_TOKEN")
APP_SECRET = os.environ.get("TW_PRS_GA_HW_ACCESS_TOKEN_SECRET")

In [4]:
url = 'https://api.twitter.com/1.1/account/verify_credentials.json'
auth = OAuth1(
            API_PUBLIC, #'USER_OAUTH_TOKEN', 
            API_SECRET, #'USER_OAUTH_TOKEN_SECRET'
            APP_PUBLIC, #'YOUR_APP_KEY', 
            APP_SECRET #'YOUR_APP_SECRET',
            )

In [5]:
# Check authorisation is working properly

In [6]:
check_auth = requests.get(url, auth=auth)

In [7]:
check_auth.reason

'OK'

In [8]:
check_auth.status_code

200

# Function 4

**Name:** `friends_of_friends`

**Returns:** list of data objects for each user that two Twitter users have in common

**Arguments:**

 - `names`: list, required; list of two Twitter users to compare friends list with
 - `keys`: list, optional; list of keys to return for information about each user.  Default value should be to return the entire data object.
 - `to_df`: bool, required; default value: False; if True, returns results in a dataframe.
 
**To Test:** We'll test your function in the following ways:

 - `friends_of_friends(['Beyonce', 'MariahCarey'])`
 - `friends_of_friends(['@Beyonce', '@MariahCarey'], to_df=True)`
 - `friends_of_friends(['Beyonce', 'MariahCarey'], keys=['id', 'name'])`
 - `friends_of_friends(['Beyonce', 'MariahCarey'], keys=['id', 'name'], to_df=True)`
 
Each of these should return 3 results. (Assuming they haven't followed the same people since this was last written).  

**Hint:** The `id` key is the unique identifier for someone, so if you want to check if two people are the same this is the best way to do it.

In [42]:
def friend_of_friends(
    names,
    keys = [], 
    to_df = False,
    auth=auth
    ):
    
    if len(names) != 2:
        print("You can only enter two names as a list.")
    
    # Checks on screen_name input
    if names[0][0] == '@':
        names[0] = names[0][1:]    

    if names[1][0] == '@':
        names[1] = names[1][1:]       

        
    # Get overall list for first name
    # Go and get results, adding users to an overall list
    friends_list_0 = []
    cursor = -1
    while cursor != 0:
        find_friends_0_url = (
                        'https://api.twitter.com/1.1/friends/list.json'
                         + "?screen_name=" + names[0]
                         + "&cursor=" + str(cursor)
                         + "&count=200" 
                        )
        friends_0_output = requests.get(find_friends_0_url, auth=auth)
        if friends_0_output.status_code == 429:
            print("Reached max API requests.\n"
                  + "Setting cursor to 0 to exit loop.\n"
                  + "friends_0s found so far will be output."
                 )
            cursor = 0
        else:
            friends_0_output_dict = friends_0_output.json()
            cursor = friends_0_output_dict['next_cursor']
            [friends_list_0.append(user) for user in friends_0_output_dict['users']]

            
    # Get overall list for second name
    friends_list_1 = []
    cursor = -1
    while cursor != 0:
        find_friends_1_url = (
                        'https://api.twitter.com/1.1/friends/list.json'
                         + "?screen_name=" + names[1]
                         + "&cursor=" + str(cursor)
                         + "&count=200" 
                        )
        friends_1_output = requests.get(find_friends_1_url, auth=auth)
        if friends_1_output.status_code == 429:
            print("Reached max API requests.\n"
                  + "Setting cursor to 0 to exit loop.\n"
                  + "friends_1s found so far will be output."
                 )
            cursor = 0
        else:
            friends_1_output_dict = friends_1_output.json()
            cursor = friends_1_output_dict['next_cursor']
            [friends_list_1.append(user) for user in friends_1_output_dict['users']]
    
    # Get list of ids for both sets of friends
    name_0_friends_ids = [i['id'] for i in friends_list_0]
    name_1_friends_ids = [i['id'] for i in friends_list_1]

    # Find common ids
    common_friends_list = []
    if len(name_0_friends_ids) < len(name_1_friends_ids):
        common_friend_ids = [i for i in name_0_friends_ids if i in name_1_friends_ids]
        common_friends_list = [i for i in friends_list_0 if i['id'] in common_friend_ids]
        
    else:
        common_friend_ids = [i for i in name_1_friends_ids if i in name_0_friends_ids]
        common_friends_list = [i for i in friends_list_1 if i['id'] in common_friend_ids]
        
    # Limit the friends_list_0 to the specific keys required
    common_friends_list_keys = []
    
    if not keys:
        common_friends_list_keys = common_friends_list
    else:
        for user in common_friends_list:
            temp_dict = {}
            for k in keys:
                temp_dict.update({k:user[k]})
            common_friends_list_keys.append(temp_dict)    
    
    # Put results into a pandas dataframe
    
    
    # Allow for pandas data frame
    if to_df == False:
        return_object = common_friends_list_keys
    elif to_df == True:
        # Loop through each potential column, and add to dictionary
        prep_df_dict = {}
        
        for k in keys:
            temp_list = [i[k] for i in common_friends_list_keys]
            prep_df_dict.update({k:temp_list})
        
        #Add data to the data frame
        user_df = pd.DataFrame(data=prep_df_dict)
        
        return_object = user_df
    else:
        return_object = 'to_df argument only accpets True or False'
                
    return return_object

In [18]:
friend_of_friends(['@Beyonce', 'MariahCarey'])

Beyonce
MariahCarey


In [9]:
url_friends_list_1 = (
    'https://api.twitter.com/1.1/friends/list.json'
    + '?screen_name=' + 'prsinkis'
    + '&count=200'
    + '&curosr=' + str(-1)
)

friends_list_sample = requests.get(url_friends_list_1, auth=auth)

In [10]:
friends_list_sample.status_code

200

In [11]:
friends_list_sample.json()

{'users': [{'id': 2584388417,
   'id_str': '2584388417',
   'name': 'HarryMTG',
   'screen_name': 'harrymtg',
   'location': 'England',
   'description': 'https://t.co/74p2Rg6Ayq | Talker @MidweekMetagame | https://t.co/WAqRVq021W | harrymtgtwitch@gmail.com | @Mana_Traders Code: HarryMTG |',
   'url': 'https://t.co/1XYJAMkvkM',
   'entities': {'url': {'urls': [{'url': 'https://t.co/1XYJAMkvkM',
       'expanded_url': 'http://www.youtube.com/c/harrymtg',
       'display_url': 'youtube.com/c/harrymtg',
       'indices': [0, 23]}]},
    'description': {'urls': [{'url': 'https://t.co/74p2Rg6Ayq',
       'expanded_url': 'http://twitch.tv/harrymtg',
       'display_url': 'twitch.tv/harrymtg',
       'indices': [0, 23]},
      {'url': 'https://t.co/WAqRVq021W',
       'expanded_url': 'http://discord.gg/tWuhYwp',
       'display_url': 'discord.gg/tWuhYwp',
       'indices': [52, 75]}]}},
   'protected': False,
   'followers_count': 1498,
   'friends_count': 187,
   'listed_count': 20,
   'crea

In [13]:
len(friends_list_sample.json()['users'])

80

In [31]:
test_b = friend_of_friends(['@Beyonce', 'MariahCarey'])

In [32]:
len(test_b)

3

In [33]:
test_b

[{'id': 40908929,
  'id_str': '40908929',
  'name': 'Usher Raymond IV',
  'screen_name': 'Usher',
  'location': '',
  'description': 'USHER: The Vegas Experience | Text me at (404) 737-1821',
  'url': 'https://t.co/6YDCXWcGDI',
  'entities': {'url': {'urls': [{'url': 'https://t.co/6YDCXWcGDI',
      'expanded_url': 'https://linktr.ee/usherofficial',
      'display_url': 'linktr.ee/usherofficial',
      'indices': [0, 23]}]},
   'description': {'urls': []}},
  'protected': False,
  'followers_count': 12186065,
  'friends_count': 564,
  'listed_count': 26144,
  'created_at': 'Mon May 18 16:36:50 +0000 2009',
  'favourites_count': 1336,
  'utc_offset': None,
  'time_zone': None,
  'geo_enabled': True,
  'verified': True,
  'statuses_count': 6041,
  'lang': None,
  'status': {'created_at': 'Thu Oct 29 16:06:41 +0000 2020',
   'id': 1321845921752125440,
   'id_str': '1321845921752125440',
   'text': 'RT @marshmellomusic: behind the scenes w/ @Usher &amp; #Imanbek \n\nToo Much 🎥 https://t.co

In [34]:
test_c = friend_of_friends(['@Beyonce', 'MariahCarey'],keys=['id','screen_name'])

In [35]:
test_c

[{'id': 40908929, 'screen_name': 'Usher'},
 {'id': 30782495, 'screen_name': 'KELLYROWLAND'},
 {'id': 18395177, 'screen_name': 'solangeknowles'}]

In [36]:
test_d = friend_of_friends(['@prsinkis', 'ickbat'],keys=['id','screen_name'])

In [37]:
test_d

[{'id': 6207162, 'screen_name': 'TheDecemberists'},
 {'id': 2011081, 'screen_name': 'rdonoghue'},
 {'id': 15439395, 'screen_name': 'stephenfry'},
 {'id': 14373916, 'screen_name': 'vfxhamilton'}]

In [38]:
test_e = friend_of_friends(['@prsinkis', 'XxdarkhorseX'],keys=['id','screen_name'])

Reached max API requests.
Setting cursor to 0 to exit loop.
friends_1s found so far will be output.


In [39]:
test_e

[{'id': 2584388417, 'screen_name': 'harrymtg'},
 {'id': 1109902312296857602, 'screen_name': 'MidweekMetagame'},
 {'id': 1095142349468913664, 'screen_name': 'KyleGbsn'},
 {'id': 854862563619688448, 'screen_name': 'Bronson190mtg'},
 {'id': 982935621969694720, 'screen_name': 'TheFactionMTG'},
 {'id': 167072801, 'screen_name': 'Gfabs5'},
 {'id': 20003820, 'screen_name': 'rbuehler'},
 {'id': 900694717, 'screen_name': 'MagicOnline'},
 {'id': 40323163, 'screen_name': 'simongoertzen'}]

In [54]:
test_f = friend_of_friends(['@Beyonce', 'MariahCarey'],keys=['id','screen_name','name','friends_count','followers_count'],to_df = True)

In [56]:
test_f

,id,screen_name,name,friends_count,followers_count
0,40908929,Usher,Usher Raymond IV,564,12186067
1,30782495,KELLYROWLAND,KELENDRIA ROWLAND,1703,6906140
2,18395177,solangeknowles,solange knowles,166,4167920


In [57]:
test_g = friend_of_friends(['@Beyonce', 'MariahCarey'],keys=['id','screen_name','name','friends_count','followers_count'],to_df = False)

Reached max API requests.
Setting cursor to 0 to exit loop.
friends_1s found so far will be output.


In [60]:
test_g

[]

# Up to here - getting an error when outputting a dataframe object

* This is becuase there is no default list of keys to build from
* So solution: go through each user, extract each level 1 key, remove duplicates, create list of keys to use as column heading
* This also means there is a need to handle errors where no value is present for that key for that user